In [24]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import random
import imageio
from torchsummary import summary
from src.MaskRCNN import MaskRCNN
from src.objectTracking import get_colour, clustering, makeTrackers


def people_only(labels, masks, boxes):
    labels, masks, boxes = np.array(labels), np.array(masks), np.array(boxes)
    people = np.where(labels == 'person', True, False)

    labels = labels[people]
    masks = masks[people]
    boxes = boxes[people]

    return labels, masks, boxes

In [30]:
model = MaskRCNN() # load the Mask-R-CNN model

v = cv2.VideoCapture('data\Arsenal_goal.mp4') # video clip to run on
ret, frame = v.read() # read first frame

masks, boxes, labels= model.predict(frame, 0.7) # use the model to detect players on the first frame
labels, masks, boxes = people_only(labels, masks, boxes) # remove non-person labels

# Identify each player's team
# obtain a vector representation of the most common colours in and around the player
colour_vectors = get_colour(frame, masks, boxes, labels) 
labels = clustering(colour_vectors, k=2) # cluster those, obtain cluster index label

multi_trackers, boxes = makeTrackers(boxes, frame)  # make multitrackers (cv2)

In [31]:
# Functions to draw the outputs 

COLOURS = [[255,0,0], [0,255,0], [0,0,255]]

def draw_classified_map(image, boxes, labels):
    names = ["ARS", "MUN"]
    team_labels = []

    for i, label in enumerate(labels):
        # apply a color mask to each object
        x, y, h, w = boxes[i]
        x1, y1, x2, y2 = int(x), int(y), int(x+h), int(y+w)

        idx = int(label)
        color = COLOURS[idx]    # CHANGE
        team_labels.append(names[idx])
        
        # draw the bounding boxes around the objects
        cv2.rectangle(image, (x1, y1), (x2, y2), color=color, 
                      thickness=2)
        # put the label text above the objects
        cv2.putText(image, names[idx], (x1, y1-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color=color, 
                    thickness=2, lineType=cv2.LINE_AA)
    return image, team_labels

# (roughly) determine where the player is standing (needs work)
def calculate_positions(boxes):
    positions = []
    for bb in boxes:
        x, y, h, w = bb
        xx = int(x + h/2)
        yy = int(y + w)
        positions.append([xx,yy])
    return positions

In [32]:
# draw the first frame
result, team_labels = draw_classified_map(frame, boxes, labels)
cv2.imshow("Frame", result)

# for saving the output
framesize = (1080, 1920)
saveVideo = True
if saveVideo: 
    out = cv2.VideoWriter('output_video.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 25, framesize)  # DOESN'T WORK :(
    output_frames = []

# loop through the remaining frames
while True:
    ret, frame = v.read()
    if not ret:
        break
    (success, boxes) = multi_trackers.update(frame)
    result, team_labels = draw_classified_map(frame, boxes, labels) # draw the BBs and labels

    positions = calculate_positions(boxes) # determines the center-bottom of the BB
    for pos in positions: # draw a circle to show those
        result = cv2.circle(result, pos, radius=10, color=(0,0,255), thickness=2)

    cv2.imshow("Frame", result)

    if saveVideo:
        frame = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
        output_frames.append(frame)

   
    key = cv2.waitKey(5) & 0xFF
    if key == ord("q"):
        break
out.release()

cv2.destroyAllWindows()

if saveVideo:
    with imageio.get_writer("output_gif.gif", mode="I") as writer: # make a GIF output
        for idx, frame in enumerate(output_frames):
            #print("Adding frame to GIF file: ", idx + 1)
            if idx%5==0: writer.append_data(frame)
    
    # FIX !!!
    for fr in output_frames: 
        out.write(fr)
    v.release() 

In [19]:
frame.shape

(1080, 1920, 3)